In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4,0, 6.0]

def forward(x):
    return x*w

def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)*(y_pred - y)

w_list = []
mse_list = []

for w in np.arange(0.0, 4.1, 0.1):
    print("w= ", w)
    l_sum = 0
    for x_val, y_val in zip(x_data, y_data):
        y_pred_val = forward(x_val)
        l = loss(x_val, y_val)
        l_sum += l
        print("\t", x_val, y_val, y_pred_val, l)
    print("MSE = ",l_sum /3)
    w_list.append(w)
    mse_list.append(l_sum/3)
    
plt.plot(w_list, mse_list)
plt.ylabel('Loss')
plt.xlabel('w')
plt.show()

In [ ]:
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = 1.0

def forwar(x):
    return x*w

def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)*(y_pred -y)

def gradient(x,y):
    return 2 * x * (x*w-y)

print("predict (before training)", 4, forward(4))

for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        grad = gradient(x_val, y_val)
        w = w - 0.01*grad
        print("\tgrad: ", x_val, y_val, round(grad,2))
        l = loss(x_val, y_val)
        
    print("progress:", epoch, "w= ", round(w, 2), "loss= ", round(l, 2))

print("predict (after training)", "4 hours", forward(4))

In [ ]:
import torch
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = torch.tensor([1.0],requires_grad = True)

def forward(x):
    return x*w

def loss(x,y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

print("predict (before training)", 4, forward(4))
for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        l = loss(x_val, y_val)
        l.backward()
        print("grad: ", x_val, y_val, w.grad.data[0])
        w.data = w.data - 0.01 * w.grad.data
        w.grad.data.zero_()
        
    print("progress:", epoch, l.data[0])
print("predict (after training)", 4, forward(4))

In [ ]:
import torch
from torch.autograd import Variable

x_data = torch.tensor([[1.0], [2.0], [3.0]])
y_data = torch.tensor([[2.0], [4.0], [6.0]])

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.linear = torch.nn.Linear(1,1)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = MyModel()

criterion = torch.nn.MSELoss(size_average = False)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(501):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    
    if(epoch%100 == 0):
        print(epoch, loss.data.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
hour_var = torch.tensor([[4.0]])
print("predict (after training)", 4, model.forward(hour_var).data[0][0])

In [ ]:
import torch
import torch.nn.functional as F

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = torch.nn.Linear(1,1)
        
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred
    
x_data = torch.Tensor([[1.0], [2.0], [3.0], [4.0]])
y_data = torch.Tensor([[0.],[0.],[1.],[1.]])

model = MyModel()
criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(1000):
    
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    if(epoch%100 == 0):
        print(epoch, loss.data.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
hour_var = torch.Tensor([[1.0]])
print("1hour : ", model(hour_var).data[0][0]>0.5)
hour_var = torch.Tensor([[7.0]])
print("7hour : ", model(hour_var).data[0][0]>0.5)

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
xy = np.loadtxt('diabetes.csv.gz', delimiter=',', dtype = np.float32)
x_data = torch.from_numpy(xy[:, 0:-1])
y_data = torch.from_numpy(xy[:, [-1]])

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.l1 = torch.nn.Linear(8,4)
        self.l2 = torch.nn.Linear(4,6)
        self.l3 = torch.nn.Linear(6,1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
    
model = MyModel()

cirterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

for epoch in range(1000):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    
    if(epoch%100 == 0):
        print("loss: ", loss)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
cnt = 0
for it in range(y_data.size()[0]):
    if((y_pred[it][0]>0.5) == y_data[it][0].type(torch.ByteTensor)):
        cnt = cnt+1
        
print(cnt)
print("Accuracy : ", cnt*100/y_data.size()[0])

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

class DiabetesDataset(Dataset):
    
    def __init__(self):
        xy = np.loadtxt('diabetes.csv.gz', delimiter = ',', dtype = np.float32)
        
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len
    
dataset = DiabetesDataset()
train_loader = DataLoader(dataset = dataset,
                         batch_size = 32,
                         shuffle = True,
                         num_workers = 0)

for epoch in range(2):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        y_pred = model(inputs)
        
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.data.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
import numpy as np
import torch

loss = torch.nn.CrossEntropyLoss()

y = torch.LongTensor([2, 0, 1])
y.requires_grad = False

y_pred1 = torch.Tensor([[2.0, 1.0, 0.1],
                      [1.0, 1.4, 0.1],
                      [1.1, 0.2, 3.1]])
y_pred2 = torch.Tensor([[0.5, 1.0, 2.5],
                      [2.2, 1.1, 0.3],
                      [1.5, 2.0, 1.3]])

l1 = loss(y_pred1, y)
l2 = loss(y_pred2, y)

print(l1, l2)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 64

train_dataset = datasets.MNIST(root='./mnist_data/',
                              train = True,
                              transform = transforms.ToTensor(),
                              download = True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                             train = False,
                             transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

class NNModel(torch.nn.Module):
    
    def __init__(self):
        super(NNModel, self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120,10)
        
    def forward(self, x):
        x = x.view(-1,784) #Flatten:(n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model = NNModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

model = model.to(device)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        optimizer.zero_grad()
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.
                  format(epoch, batch_idx* len(data), len(train_loader.dataset), 
                         100. * batch_idx / len(train_loader), loss.data.item()))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        test_loss += criterion(output, target).data.item()
        
        pred = output.data.max(1, keepdim = True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.
         format(test_loss, correct, len(test_loader.dataset),
               100.*correct / len(test_loader.dataset)))
    
    
for epoch in range(1, 9):
    train(epoch)
    test()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64

train_dataset = datasets.MNIST(root = './mnist_data/', train = True, transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root = './mnist_data', train = False, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

class InceptionModule(nn.Module):
    
    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size = 5, padding = 2)
        
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size = 3, padding = 1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size = 3, padding = 1)
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size = 1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x, kernel_size = 3, stride =1, padding =1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        
        #torch.cat
        #x = tensor([[0.6580, -1.0969, -0.4614],
        #[-0.1034, -0.5790, 0.1497]])
        #torch.cat((x,x,x),0) -> (6, 3) 행렬
        #torch.cat((x,x,x),1) -> (2, 9) 행렬로 합쳐짐
        #(n*n, 16), (n*n, 24), (n*n, 24), (n*n, 24) -> (n*n, 88)
        
        return torch.cat(outputs, 1)
    
class MainNet(nn.Module):
    
    def __init__(self):
        super(MainNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(88,20,kernel_size = 5)
        
        self.incept1 = InceptionModule(in_channels = 10)
        self.incept2 = InceptionModule(in_channels = 20)
        
        self.max_pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)
        
    def forward(self, x):
        in_size = x.size(0) #size(0) -> (n, 28 * 28) return n
        x = F.relu(self.max_pool(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.max_pool(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)
    
model = MainNet().to(device)

optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.NLLLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epcoh : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format
                 (epoch, batch_idx * len(data), len(train_loader.dataset),
                 100. * batch_idx / len(train_loader), loss.data.item()))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)/batch_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
         (test_loss, correct, len(test_loader.dataset),
         100.*correct, len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()